In [1]:
import pandas as pd

weather = pd.read_csv('../data/weather.csv')
price = pd.read_csv('../data/all_pricedata.csv')

weather.timestamp = weather.timestamp.apply(pd.Timestamp)
price['timestamp'] = price['MTU (CET/CEST)'].apply(lambda ts: pd.Timestamp(ts.split('-')[0].strip()))

merged = price.merge(weather, how = 'inner', on = ['timestamp', 'timestamp'])


In [2]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [3]:
merged

,Unnamed: 0,MTU (CET/CEST),Day-ahead Price [EUR/MWh],Currency,BZN|FI,timestamp,cloud_amount,pressure,percipitation_amount,relative_humidity,precipitation_intensity,snow_depth,air_temperature,dew_point_temperature,horizontal_visibility,wind_direction,gust_speed,wind_speed
0,0,01.01.2015 00:00 - 01.01.2015 01:00,27.38,EUR,NaN,2015-01-01 00:00:00,7.0,1010.7,0.0,89.0,0.0,10.0,3.3,1.6,13700.0,239.0,7.1,4.4
1,1,01.01.2015 01:00 - 01.01.2015 02:00,23.37,EUR,NaN,2015-01-01 01:00:00,7.0,1010.5,0.0,89.0,0.0,9.0,3.2,1.5,17390.0,234.0,6.5,4.6
2,2,01.01.2015 02:00 - 01.01.2015 03:00,19.33,EUR,NaN,2015-01-01 02:00:00,5.0,1010.5,0.0,90.0,0.0,10.0,3.0,1.6,18170.0,231.0,6.3,4.3
3,3,01.01.2015 03:00 - 01.01.2015 04:00,17.66,EUR,NaN,2015-01-01 03:00:00,7.0,1010.1,0.0,91.0,0.0,9.0,2.6,1.3,18620.0,220.0,6.4,4.2
4,4,01.01.2015 04:00 - 01.01.2015 05:00,17.53,EUR,NaN,2015-01-01 04:00:00,7.0,1009.7,0.0,89.0,0.0,9.0,3.3,1.7,30320.0,216.0,5.4,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52923,8781,31.12.2020 20:00 - 31.12.2020 21:00,24.84,EUR,NaN,2020-12-31 20:00:00,7.0,1008.7,0.3,99.0,0.0,0.0,0.7,0.5,6000.0,225.0,5.2,3.5
52924,8782,31.12.2020 21:00 - 31.12.2020 22:00,28.0,EUR,NaN,2020-12-31 21:00:00,1.0,1009.0,0.0,98.0,0.0,0.0,0.0,-0.2,2500.0,254.0,3.2,2.3
52925,8783,31.12.2020 22:00 - 31.12.2020 23:00,26.55,EUR,NaN,2020-12-31 22:00:00,0.0,1009.3,0.0,99.0,0.0,0.0,-0.6,-0.8,590.0,204.0,4.2,2.8
52926,8784,31.12.2020 23:00 - 01.01.2021 00:00,24.07,EUR,NaN,2020-12-31 23:00:00,9.0,1010.0,0.0,99.0,0.0,0.0,-0.4,-0.5,290.0,237.0,2.3,1.4


In [4]:
merged['doy'] = merged['timestamp'].apply(lambda x: x.day_of_year)
merged['hod'] = merged['timestamp'].apply(lambda x: x.hour)
merged

,Unnamed: 0,MTU (CET/CEST),Day-ahead Price [EUR/MWh],Currency,BZN|FI,timestamp,cloud_amount,pressure,percipitation_amount,relative_humidity,precipitation_intensity,snow_depth,air_temperature,dew_point_temperature,horizontal_visibility,wind_direction,gust_speed,wind_speed,doy,hod
0,0,01.01.2015 00:00 - 01.01.2015 01:00,27.38,EUR,NaN,2015-01-01 00:00:00,7.0,1010.7,0.0,89.0,0.0,10.0,3.3,1.6,13700.0,239.0,7.1,4.4,1,0
1,1,01.01.2015 01:00 - 01.01.2015 02:00,23.37,EUR,NaN,2015-01-01 01:00:00,7.0,1010.5,0.0,89.0,0.0,9.0,3.2,1.5,17390.0,234.0,6.5,4.6,1,1
2,2,01.01.2015 02:00 - 01.01.2015 03:00,19.33,EUR,NaN,2015-01-01 02:00:00,5.0,1010.5,0.0,90.0,0.0,10.0,3.0,1.6,18170.0,231.0,6.3,4.3,1,2
3,3,01.01.2015 03:00 - 01.01.2015 04:00,17.66,EUR,NaN,2015-01-01 03:00:00,7.0,1010.1,0.0,91.0,0.0,9.0,2.6,1.3,18620.0,220.0,6.4,4.2,1,3
4,4,01.01.2015 04:00 - 01.01.2015 05:00,17.53,EUR,NaN,2015-01-01 04:00:00,7.0,1009.7,0.0,89.0,0.0,9.0,3.3,1.7,30320.0,216.0,5.4,3.7,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52923,8781,31.12.2020 20:00 - 31.12.2020 21:00,24.84,EUR,NaN,2020-12-31 20:00:00,7.0,1008.7,0.3,99.0,0.0,0.0,0.7,0.5,6000.0,225.0,5.2,3.5,366,20
52924,8782,31.12.2020 21:00 - 31.12.2020 22:00,28.0,EUR,NaN,2020-12-31 21:00:00,1.0,1009.0,0.0,98.0,0.0,0.0,0.0,-0.2,2500.0,254.0,3.2,2.3,366,21
52925,8783,31.12.2020 22:00 - 31.12.2020 23:00,26.55,EUR,NaN,2020-12-31 22:00:00,0.0,1009.3,0.0,99.0,0.0,0.0,-0.6,-0.8,590.0,204.0,4.2,2.8,366,22
52926,8784,31.12.2020 23:00 - 01.01.2021 00:00,24.07,EUR,NaN,2020-12-31 23:00:00,9.0,1010.0,0.0,99.0,0.0,0.0,-0.4,-0.5,290.0,237.0,2.3,1.4,366,23


In [5]:
dataset = merged[['pressure', 'wind_speed', 'air_temperature', 'doy', 'hod', 'Day-ahead Price [EUR/MWh]']]
dataset = dataset.dropna()
X = dataset[['pressure', 'wind_speed', 'air_temperature', 'doy', 'hod']]
y = dataset['Day-ahead Price [EUR/MWh]']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), svm.SVR(C=1.0, epsilon=0.3))
regr.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.3))])

In [8]:
X_test.head()

,pressure,wind_speed,air_temperature,doy,hod
25879,1001.9,5.5,17.3,193,18
20291,1027.0,8.1,3.1,110,8
43964,1016.2,3.0,-0.1,360,19
22135,1011.4,3.2,5.8,127,17
19684,1016.9,4.1,-1.1,85,4


In [9]:
y_test.head()

25879    33.61
20291     40.5
43964     38.9
22135    40.83
19684    26.62
Name: Day-ahead Price [EUR/MWh], dtype: object

In [11]:
regr.score(X_test, y_test)

0.22257698775872203